In [1]:
import sys

import numpy as np


import argparse

import os

current_script_path = os.getcwd()
main_folder_path = os.path.dirname(current_script_path)

if main_folder_path not in sys.path:
    sys.path.append(main_folder_path)

from tqdm import tqdm 

import numpy as np
import matplotlib.pyplot as plt
import pykonal
import random

from scipy.io import loadmat




In [2]:
maps = np.load("mazes_15_2_3000.npz")
maps = 1 - maps["maps"]


In [3]:
# Number of trials
# train_trial = 5054

train_trial = 2000
test_trial = 1000

num_trials = train_trial+test_trial

goal_trials = 20
env_size = 15
count = 0

x_values_array = np.zeros((env_size * env_size * 1, num_trials * goal_trials))
y_values_array = np.zeros((env_size * env_size * 1, num_trials * goal_trials))
velocity_matrices_array = np.ones((num_trials * goal_trials, env_size, env_size))
travel_time_values_array = np.zeros((num_trials * goal_trials, env_size, env_size))
goals = np.zeros((num_trials * goal_trials, 2))

for trial in tqdm(range(num_trials)):
    
    im_np = maps[trial,:,:]
    original_maze = im_np
    condition1 = original_maze == 1
    row_indices, col_indices = np.indices(original_maze.shape)
    condition2 = (row_indices < env_size) & (col_indices < env_size)
    combined_condition = condition1 & condition2

    for goal_trial in range(goal_trials):
        passable_indices = np.argwhere(combined_condition)
        high_values_mask = None
        if passable_indices.size > 0:
            if(trial * goal_trials + goal_trial!=count):
                break
            while np.all(travel_time_values_array[trial * goal_trials + goal_trial, :, :] == 0):
                velocity_matrix = np.array(original_maze)
                goal_index = random.choice(passable_indices)
                goal = goal_index[0], goal_index[1]
                # goal = 7,7
                # print(goal)
                # Set up the Eikonal solver
                solver = pykonal.EikonalSolver(coord_sys="cartesian")
                solver.velocity.min_coords = 0, 0, 0
                solver.velocity.node_intervals = 1, 1, 1
                solver.velocity.npts = env_size, env_size, 1
                solver.velocity.values = velocity_matrix.reshape(env_size, env_size, 1)
                src_idx = goal[0], goal[1], 0
                solver.traveltime.values[src_idx] = 0
                solver.unknown[src_idx] = False
                solver.trial.push(*src_idx)
                solver.solve()
                velocity_matrices_array[trial * goal_trials + goal_trial, :, :] = velocity_matrix
                goals[trial * goal_trials + goal_trial,:] = goal
                travel_time_values_array[trial * goal_trials + goal_trial, :, :] = solver.traveltime.values[:, :, 0]
                high_values_mask = solver.traveltime.values[:, :, 0] > 1000
                input_mask = (velocity_matrix == 0)
                travel_time_values_array[trial * goal_trials + goal_trial, high_values_mask] = 0
            if(high_values_mask!=input_mask).any():
                velocity_matrices_array[trial * goal_trials + goal_trial, high_values_mask] = 0

            if np.any(travel_time_values_array[trial * goal_trials + goal_trial, :, :] > 1000):
                print("Improper Data Created")
                break
            if np.all(travel_time_values_array[trial * goal_trials + goal_trial, :, :] == 0):
                print("Improper Data Created")
                break
        else:
            print("Improper Data Created")
            break

        count+=1 



  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [00:07<00:00, 379.26it/s]


In [4]:
np.save("multgoal_dataset/goals.npy",goals)

# Save velocity_matrices_array as "mask.npy"
np.save("multgoal_dataset/mask.npy", velocity_matrices_array)

# Save travel_time_values_array as "output.npy"
np.save("multgoal_dataset/output.npy", travel_time_values_array)

In [5]:
def calculate_signed_distance(velocity_matrix):
    # Find the indices of obstacles
    obstacle_indices = np.where(velocity_matrix == 0)

    # Check if there are no obstacles
    if obstacle_indices[0].size == 0:
        # Handle the case when there are no obstacles
        return np.zeros_like(velocity_matrix)

    # Create a meshgrid of indices
    i, j = np.meshgrid(np.arange(velocity_matrix.shape[1]), np.arange(velocity_matrix.shape[0]), indexing='ij')  # Note the change in the order of indices
    
    # Calculate distances to obstacles using vectorized operations
    distances = np.sqrt((obstacle_indices[0][:, None, None] - i[None, :, :])**2 + (obstacle_indices[1][:, None, None] - j[None, :, :])**2)

    # Find the minimum distance for each point
    signed_distance = np.min(distances, axis=0)
    return signed_distance


In [6]:

signed_distance_array = np.zeros((num_trials*goal_trials, env_size, env_size))

for i in tqdm(range(num_trials*goal_trials)):
    signed_distance_array[i,:,:] = calculate_signed_distance(velocity_matrices_array[i,:,:].reshape(env_size,env_size))


# Save signed_distance_array as "dist_in.npy"
# np.save("test_dist_in.npy",signed_distance_array)
np.save("multgoal_dataset/dist_in.npy", signed_distance_array)

  0%|          | 0/60000 [00:00<?, ?it/s]

100%|██████████| 60000/60000 [00:05<00:00, 11144.34it/s]
